# Task 7: AutoFeatureSelector Tool
## This task is to test your understanding of various Feature Selection methods outlined in the lecture and the ability to apply this knowledge in a real-world dataset to select best features and also to build an automated feature selection tool as your toolkit

### Use your knowledge of different feature selector methods to build an Automatic Feature Selection tool
- Pearson Correlation
- Chi-Square
- RFE
- Embedded
- Tree (Random Forest)
- Tree (Light GBM)

### Dataset: FIFA 19 Player Skills
#### Attributes: FIFA 2019 players attributes like Age, Nationality, Overall, Potential, Club, Value, Wage, Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight, LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
from collections import Counter
import math
from scipy import stats

In [2]:
player_df = pd.read_csv("fifa19.csv")

In [3]:
numcols = ['Overall', 'Crossing','Finishing',  'ShortPassing',  'Dribbling','LongPassing', 'BallControl', 'Acceleration','SprintSpeed', 'Agility',  'Stamina','Volleys','FKAccuracy','Reactions','Balance','ShotPower','Strength','LongShots','Aggression','Interceptions']
catcols = ['Preferred Foot','Position','Body Type','Nationality','Weak Foot']

In [4]:
player_df = player_df[numcols+catcols]

In [5]:
player_df

,Overall,Crossing,Finishing,ShortPassing,Dribbling,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,...,ShotPower,Strength,LongShots,Aggression,Interceptions,Preferred Foot,Position,Body Type,Nationality,Weak Foot
0,94,84.0,95.0,90.0,97.0,87.0,96.0,91.0,86.0,91.0,...,85.0,59.0,94.0,48.0,22.0,Left,RF,Messi,Argentina,4.0
1,94,84.0,94.0,81.0,88.0,77.0,94.0,89.0,91.0,87.0,...,95.0,79.0,93.0,63.0,29.0,Right,ST,C. Ronaldo,Portugal,4.0
2,92,79.0,87.0,84.0,96.0,78.0,95.0,94.0,90.0,96.0,...,80.0,49.0,82.0,56.0,36.0,Right,LW,Neymar,Brazil,5.0
3,91,17.0,13.0,50.0,18.0,51.0,42.0,57.0,58.0,60.0,...,31.0,64.0,12.0,38.0,30.0,Right,GK,Lean,Spain,3.0
4,91,93.0,82.0,92.0,86.0,91.0,91.0,78.0,76.0,79.0,...,91.0,75.0,91.0,76.0,61.0,Right,RCM,Normal,Belgium,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,47,34.0,38.0,49.0,42.0,45.0,43.0,54.0,57.0,60.0,...,43.0,47.0,38.0,46.0,46.0,Right,CM,Lean,England,2.0
18203,47,23.0,52.0,43.0,39.0,25.0,40.0,41.0,39.0,38.0,...,41.0,67.0,42.0,47.0,16.0,Right,ST,Normal,Sweden,2.0
18204,47,25.0,40.0,38.0,45.0,28.0,44.0,70.0,69.0,50.0,...,45.0,32.0,45.0,32.0,15.0,Right,ST,Normal,England,3.0
18205,47,44.0,50.0,42.0,51.0,32.0,52.0,61.0,60.0,52.0,...,64.0,48.0,34.0,33.0,22.0,Right,RW,Lean,England,3.0


In [6]:
traindf = pd.concat([player_df[numcols], pd.get_dummies(player_df[catcols])],axis=1)
features = traindf.columns

traindf = traindf.dropna()

In [7]:
traindf = pd.DataFrame(traindf,
                       columns=features)

In [8]:
y = traindf['Overall']>=87
X = traindf.copy()
del X['Overall']

In [9]:
X.head()

,Crossing,Finishing,ShortPassing,Dribbling,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Stamina,...,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekistan,Nationality_Venezuela,Nationality_Wales,Nationality_Zambia,Nationality_Zimbabwe
0,84.0,95.0,90.0,97.0,87.0,96.0,91.0,86.0,91.0,72.0,...,0,0,0,0,0,0,0,0,0,0
1,84.0,94.0,81.0,88.0,77.0,94.0,89.0,91.0,87.0,88.0,...,0,0,0,0,0,0,0,0,0,0
2,79.0,87.0,84.0,96.0,78.0,95.0,94.0,90.0,96.0,81.0,...,0,0,0,0,0,0,0,0,0,0
3,17.0,13.0,50.0,18.0,51.0,42.0,57.0,58.0,60.0,43.0,...,0,0,0,0,0,0,0,0,0,0
4,93.0,82.0,92.0,86.0,91.0,91.0,78.0,76.0,79.0,90.0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
len(X.columns)

223

### Set some fixed set of features

In [11]:
feature_name = list(X.columns)
# no of maximum features we need to select
num_feats=30

## Filter Feature Selection - Pearson Correlation
# Checking the correlation and sort it to get top 30 features.

### Pearson Correlation function

In [12]:
def cor_selector(X, y,num_feats):
    # Your code goes here (Multiple lines)
    cor_list = []
    feature_name = X.columns.tolist()
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
        
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    cor_support = [True if i in cor_feature else False for i in feature_name]
    # Your code ends here
    return cor_support, cor_feature

In [13]:
cor_support, cor_feature = cor_selector(X, y,num_feats)
print(str(len(cor_feature)), 'selected features')

30 selected features


### List the selected features from Pearson Correlation

In [14]:
cor_feature

['Nationality_Costa Rica',
 'Position_LAM',
 'Nationality_Uruguay',
 'Acceleration',
 'SprintSpeed',
 'Strength',
 'Nationality_Gabon',
 'Nationality_Slovenia',
 'Stamina',
 'Weak Foot',
 'Agility',
 'Crossing',
 'Nationality_Belgium',
 'Dribbling',
 'ShotPower',
 'LongShots',
 'Finishing',
 'BallControl',
 'FKAccuracy',
 'LongPassing',
 'Volleys',
 'ShortPassing',
 'Position_RF',
 'Position_LF',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Body Type_Courtois',
 'Body Type_Neymar',
 'Body Type_Messi',
 'Body Type_C. Ronaldo',
 'Reactions']

## Filter Feature Selection - Chi-Sqaure
# Select the features that has maximum chi2 values

In [15]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

### Chi-Squared Selector function

In [16]:
def chi_squared_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    X_norm = MinMaxScaler().fit_transform(X)
    chi_selector = SelectKBest(chi2, k=num_feats)
    chi_selector.fit(X_norm, y)
    chi_support = chi_selector.get_support()
    chi_feature = X.loc[:, chi_support].columns.tolist()
    # Your code ends here
    return chi_support, chi_feature

In [17]:
chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
print(str(len(chi_feature)), 'selected features')

30 selected features


### List the selected features from Chi-Square 

In [18]:
chi_feature

['Finishing',
 'ShortPassing',
 'LongPassing',
 'BallControl',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'LongShots',
 'Position_CM',
 'Position_LAM',
 'Position_LF',
 'Position_LW',
 'Position_RB',
 'Position_RF',
 'Body Type_C. Ronaldo',
 'Body Type_Courtois',
 'Body Type_Messi',
 'Body Type_Neymar',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Nationality_Belgium',
 'Nationality_Costa Rica',
 'Nationality_Croatia',
 'Nationality_Egypt',
 'Nationality_England',
 'Nationality_France',
 'Nationality_Gabon',
 'Nationality_Slovakia',
 'Nationality_Slovenia',
 'Nationality_Spain',
 'Nationality_Uruguay']

## Wrapper Feature Selection - Recursive Feature Elimination
# RFE select features by recursively, the least important feature are cut from the current set of features.

In [19]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

### RFE Selector function

In [20]:
def rfe_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    X_norm = MinMaxScaler().fit_transform(X)
    rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats,
                       step=10, verbose=5)
    
    rfe_selector.fit(X_norm,y)
    rfe_support = rfe_selector.get_support()
    rfe_feature = X.loc[:, rfe_support].columns.tolist()
    # Your code ends here
    return rfe_support, rfe_feature

In [21]:
rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
print(str(len(rfe_feature)), 'selected features')

Fitting estimator with 223 features.
Fitting estimator with 213 features.
Fitting estimator with 203 features.
Fitting estimator with 193 features.
Fitting estimator with 183 features.
Fitting estimator with 173 features.
Fitting estimator with 163 features.
Fitting estimator with 153 features.
Fitting estimator with 143 features.
Fitting estimator with 133 features.
Fitting estimator with 123 features.
Fitting estimator with 113 features.
Fitting estimator with 103 features.
Fitting estimator with 93 features.
Fitting estimator with 83 features.
Fitting estimator with 73 features.
Fitting estimator with 63 features.
Fitting estimator with 53 features.
Fitting estimator with 43 features.
Fitting estimator with 33 features.
30 selected features


### List the selected features from RFE

In [22]:
rfe_feature

['Finishing',
 'ShortPassing',
 'LongPassing',
 'BallControl',
 'SprintSpeed',
 'Agility',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'Strength',
 'Weak Foot',
 'Position_CAM',
 'Position_CM',
 'Position_GK',
 'Position_LCB',
 'Position_LM',
 'Position_RB',
 'Position_RCB',
 'Position_RF',
 'Position_RM',
 'Position_RW',
 'Body Type_Courtois',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Nationality_Belgium',
 'Nationality_Costa Rica',
 'Nationality_Croatia',
 'Nationality_Gabon',
 'Nationality_Netherlands',
 'Nationality_Slovenia',
 'Nationality_Uruguay']

## Embedded Selection - Lasso: SelectFromModel
# L1: adding penalty to parameters of the model to reduce the freedom of models => avoiding over-fitting

In [23]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [24]:
def embedded_log_reg_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    X_norm = MinMaxScaler().fit_transform(X)
    embedded_lr_selector = SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear'), max_features=num_feats)
    embedded_lr_selector.fit(X_norm,y)

    embedded_lr_support = embedded_lr_selector.get_support()
    embedded_lr_feature = X.loc[:, embedded_lr_support].columns.tolist()
    # Your code ends here
    return embedded_lr_support, embedded_lr_feature

In [25]:
embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
print(str(len(embedded_lr_feature)), 'selected features')


27 selected features


In [26]:
embedded_lr_feature

['LongPassing',
 'Reactions',
 'Balance',
 'Aggression',
 'Preferred Foot_Right',
 'Position_CAM',
 'Position_CM',
 'Position_GK',
 'Position_LCB',
 'Position_LM',
 'Position_LW',
 'Position_RB',
 'Position_RCB',
 'Position_RW',
 'Body Type_Lean',
 'Body Type_Stocky',
 'Nationality_Belgium',
 'Nationality_Brazil',
 'Nationality_Croatia',
 'Nationality_England',
 'Nationality_France',
 'Nationality_Germany',
 'Nationality_Italy',
 'Nationality_Netherlands',
 'Nationality_Portugal',
 'Nationality_Slovenia',
 'Nationality_Uruguay']

## Tree based(Random Forest): SelectFromModel

In [27]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [28]:
def embedded_rf_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    embedded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100),max_features=num_feats)
    embedded_rf_selector.fit(X,y)

    embedded_rf_support = embedded_rf_selector.get_support()
    embedded_rf_feature = X.loc[:, embedded_rf_support].columns.tolist()
    # Your code ends here
    return embedded_rf_support, embedded_rf_feature

In [29]:
embedder_rf_support, embedder_rf_feature = embedded_rf_selector(X, y, num_feats)
print(str(len(embedder_rf_feature)), 'selected features')

21 selected features


In [30]:
embedder_rf_feature

['Crossing',
 'Finishing',
 'ShortPassing',
 'Dribbling',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Stamina',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'Balance',
 'ShotPower',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Weak Foot',
 'Body Type_Courtois']

## Tree based(Light GBM): SelectFromModel

In [31]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [32]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

In [33]:
def embedded_lgbm_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    lgbc = LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
                         reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)
    embedded_lgbm_selector = SelectFromModel(lgbc, max_features=num_feats)
    embedded_lgbm_selector.fit(X,y)
    
    embedded_lgbm_support = embedded_lgbm_selector.get_support()
    embedded_lgbm_feature = X.loc[:,embedded_lgbm_support].columns.tolist()
    # Your code ends here
    return embedded_lgbm_support, embedded_lgbm_feature

In [34]:
# embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)
# print(str(len(embeded_lgb_feature)), 'selected features')

In [35]:
# embedded_lgbm_feature

## Putting all of it together: AutoFeatureSelector Tool

In [38]:
pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_log_reg_selector,
                                    'Random Forest':embedded_rf_selector, 'LightGBM':embedded_lgbm_selector})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(num_feats)

/Users/hau.nguyen2904/anaconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,Volleys,True,True,True,<function embedded_log_reg_selector at 0x1664d...,<function embedded_rf_selector at 0x16a3d3490>,<function embedded_lgbm_selector at 0x29ab4d750>,3
2,ShortPassing,True,True,True,<function embedded_log_reg_selector at 0x1664d...,<function embedded_rf_selector at 0x16a3d3490>,<function embedded_lgbm_selector at 0x29ab4d750>,3
3,Reactions,True,True,True,<function embedded_log_reg_selector at 0x1664d...,<function embedded_rf_selector at 0x16a3d3490>,<function embedded_lgbm_selector at 0x29ab4d750>,3
4,Position_RF,True,True,True,<function embedded_log_reg_selector at 0x1664d...,<function embedded_rf_selector at 0x16a3d3490>,<function embedded_lgbm_selector at 0x29ab4d750>,3
5,Nationality_Uruguay,True,True,True,<function embedded_log_reg_selector at 0x1664d...,<function embedded_rf_selector at 0x16a3d3490>,<function embedded_lgbm_selector at 0x29ab4d750>,3
6,Nationality_Slovenia,True,True,True,<function embedded_log_reg_selector at 0x1664d...,<function embedded_rf_selector at 0x16a3d3490>,<function embedded_lgbm_selector at 0x29ab4d750>,3
7,Nationality_Gabon,True,True,True,<function embedded_log_reg_selector at 0x1664d...,<function embedded_rf_selector at 0x16a3d3490>,<function embedded_lgbm_selector at 0x29ab4d750>,3
8,Nationality_Costa Rica,True,True,True,<function embedded_log_reg_selector at 0x1664d...,<function embedded_rf_selector at 0x16a3d3490>,<function embedded_lgbm_selector at 0x29ab4d750>,3
9,Nationality_Belgium,True,True,True,<function embedded_log_reg_selector at 0x1664d...,<function embedded_rf_selector at 0x16a3d3490>,<function embedded_lgbm_selector at 0x29ab4d750>,3
10,LongPassing,True,True,True,<function embedded_log_reg_selector at 0x1664d...,<function embedded_rf_selector at 0x16a3d3490>,<function embedded_lgbm_selector at 0x29ab4d750>,3


## Can you build a Python script that takes dataset and a list of different feature selection methods that you want to try and output the best (maximum votes) features from all methods?

In [ ]:
def preprocess_dataset(dataset_path):
    # Your code starts here (Multiple lines)
    df = pd.read_csv(dataset_path)
    
    #define num & cat columns
    num_col = df.select_dtypes(include=np.number)
    num_col_list = df.select_dtypes(include=['float64','int64']).columns.tolist()
    
    cat_col = df.select_dtypes(exclude=np.number)
    cat_col_list = df.select_dtypes(include=['object']).columns.tolist()
    
    
    df = df[num_col_list+cat_col_list]
    
    traindf = pd.concat([df[num_col_list], pd.get_dummies(df[cat_col_list])],axis=1)
    features = traindf.columns

    traindf = traindf.dropna()
    
    traindf = pd.DataFrame(traindf,
                       columns=features)
    
    
    label = input("Enter a dependent feature: ")
    
    y = traindf[label]
    X = traindf.drop(label,axis=1)
    
    num_feats = input("Enter number of features you want to get: ")
    
    # Your code ends here
    return X, y, num_feats

In [ ]:
def autoFeatureSelector(dataset_path, methods=[]):
    # Parameters
    # data - dataset to be analyzed (csv file)
    # methods - various feature selection methods we outlined before, use them all here (list)
    
    # preprocessing
    X, y, num_feats = preprocess_dataset(dataset_path)
    
    # Run every method we outlined above from the methods list and collect returned best features from every method
    if 'pearson' in methods:
        cor_support, cor_feature = cor_selector(X, y,num_feats)
    if 'chi-square' in methods:
        chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
    if 'rfe' in methods:
        rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
    if 'log-reg' in methods:
        embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
    if 'rf' in methods:
        embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X, y, num_feats)
    if 'lgbm' in methods:
        embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)
    
    
    # Combine all the above feature list and count the maximum set of features that got selected by all methods
    #### Your Code starts here (Multiple lines)
    df = pd.DataFrame({'Feature': cor_feature+chi_feature+rfe_feature+embedded_lr_feature+embedded_rf_feature+embedded_lgbm_feature})

    best_features = df.value_counts()[df.value_counts() == 6]
    
    #### Your Code ends here
    return best_features

In [ ]:
best_features = autoFeatureSelector(dataset_path="data/fifa19.csv", methods=['pearson', 'chi-square', 'rfe', 'log-reg', 'rf', 'lgbm'])
best_features

### Last, Can you turn this notebook into a python script, run it and submit the python (.py) file that takes dataset and list of methods as inputs and outputs the best features